In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE40589"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE40589"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE40589.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE40589.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE40589.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE40589_family.soft.gz', 'GSE40589_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE40589/GSE40589_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE40589/GSE40589_series_matrix.txt.gz


Background Information:
!Series_title	"Gene expression in human subcutaneous adipose tissue after administration of Gastric Insulinotropic Peptide (GIP)"
!Series_summary	"The intention of this clinical study was to investigate the effect of GIP administration for 240 min on gene expression in human subcutaneous adipose tissue. Three conditions have been tersted: 1. Sole infusion of GIP or NaCl as control; 2. GIP or NaCl administration under euglycemic-hyperinsulinemic clamp conditions; 3. GIP or NaCl administration under hyperglycemic-hyperinsulinemic clamp conditions to mimic the postprandial state."
!Series_overall_design	"In each participant a complete physical examination and evaluation of medical history was performed, including an oral glucose tolerance test (oGTT) with 75 g glucose after overnight fast to ensure the metabolic state. Standard fasting laboratory and clinical chemistry evaluations were done. Synthetic human GIP (1-42) was dissolved in saline (0.9% NaCl-solution) un

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Optional, Callable, Dict, Any
import json
import re

# 1. Gene Expression Data Availability
# From the background information, this dataset involves gene expression in human subcutaneous adipose tissue
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary, we don't see trait (metabolic rate)
# specific data in the limited information provided.
# While the study is about metabolic effects, there's no explicit measurement of metabolic rate
# in the available sample characteristics
trait_row = None
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> Optional[float]:
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Based on the study design, different conditions represent different metabolic states
    # But we don't have access to this information in the current sample characteristics
    return None

def convert_age(value: str) -> Optional[float]:
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Convert to binary (0 for female, 1 for male)
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (no clinical data available for trait)
# We're not attempting to read from a file as the necessary data is not available


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 41000
First 20 gene/probe identifiers:
Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers from the previous step
# These identifiers (like 'A_23_P100001') appear to be probe IDs from an Agilent microarray platform,
# not standard human gene symbols.
# Agilent probe IDs follow this pattern where A_23_P refers to the platform and the numbers identify specific probes.
# These need to be mapped to actual gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns corresponding to gene identifiers and gene symbols in the annotation
# From the preview, 'ID' contains the probe identifiers which match the gene expression data indices
# 'GENE_SYMBOL' contains the human gene symbols we want to map to
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# 2. Get the gene mapping dataframe using the function from the library
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print some information about the result
print("\nMapping information:")
print(f"Number of unique probes in mapping: {len(mapping_df['ID'].unique())}")
print(f"Number of unique genes mapped to: {len(gene_data.index)}")
print("\nFirst 10 gene symbols after mapping:")
print(gene_data.index[:10])



Mapping information:
Number of unique probes in mapping: 30936
Number of unique genes mapped to: 18488

First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# From the examination of the sample characteristics, we can see that:
# - There is no metabolic rate data available
# - The only feature is "tissue: subcutaneous adipose tissue" which is constant

# Create an empty DataFrame for clinical data since no relevant trait data exists
clinical_df = pd.DataFrame()

# Save the gene data as it's the only usable component
print("No clinical data found for Metabolic_Rate in this dataset.")

# Since we don't have trait data, we can't link the clinical and genetic data
# We'll create an empty DataFrame for linked_data
linked_data = pd.DataFrame()

# 5. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from subcutaneous adipose tissue but lacks metabolic rate measurements. The study examines GIP administration effects on gene expression under different conditions, but no direct metabolic rate measurements are available."

# Create an empty DataFrame for final validation
empty_df = pd.DataFrame()

# Validate and save cohort info with is_trait_available=False
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,
    is_biased=False,  # Set to False since we can't evaluate bias without trait data
    df=empty_df,
    note=note
)

print("Dataset validation completed. Gene data saved, but no clinical or linked data available for this trait.")

Shape of gene data after normalization: (18247, 96)


Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE40589.csv
Sample characteristics dictionary:
{0: ['tissue: subcutaneous adipose tissue']}
No clinical data found for Metabolic_Rate in this dataset.
Abnormality detected in the cohort: GSE40589. Preprocessing failed.
Dataset validation completed. Gene data saved, but no clinical or linked data available for this trait.
